In [ ]:
# Get (start, end) index of tables inside the text file 
def find_index(text, text_start, text_end_1, text_end_2=None):
    start_index, end_index = [], []
    start, end = 0, 0
    for i in range(50):
        start = text.find(text_start, start+1)
        end_of_report = end
        end = text.find(text_end_1, end+1)
        if end == -1:
            if text_end_2 == None: break
            end = text.find(text_end_2, end_of_report+1)
            start_index.append(start)
            end_index.append(end)
            break
        else: 
            start = start
            end = end
            start_index.append(start)
            end_index.append(end)
    return start_index, end_index

In [ ]:
import os
import pandas as pd
import numpy as np

# Get folder directory that contains all text files
path = "/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/statement/"
dir_list = os.listdir(path)

df_combine = pd.DataFrame()
#Loop through each text file
for file_dir in dir_list:
    
    # Open file
    with open(path + file_dir,"rb") as f:
        text_all = f.read()
        text_all = text_all.decode("utf-16")

    start_account_index_list, end_account_index_list = find_index(text = text_all,
                                                                    text_start = "LOAN A/C NO:",
                                                                    text_end_1 = "\r\n                                                                     ** End Of Report **")

    for start_account_index, end_account_index in zip(start_account_index_list, end_account_index_list):
        text_acc = text_all[start_account_index : end_account_index]
        start_table_index_list, end_table_index_list = find_index(text = text_acc,
                                                                    text_start = "=\r",
                                                                    text_end_1 = "\r\nDate printed",
                                                                    text_end_2 = "\r\n                                                                     ** End Of Report **")
        
        extract_array = list()
        for start_table_index, end_table_index in zip(start_table_index_list, end_table_index_list):
            # Get (start, end) index of tables inside the text file
            text_table = text_acc[start_table_index : end_table_index]
            text_table = text_table.replace("\r", "")
            text_table = text_table.replace("-", "")
            text_table = text_table.split("\n")
            text_table = text_table[1:]

            # Extract text to list
            for text_array in text_table:
                list_split = text_array.split(" ")
                while "" in list_split:
                    list_split.remove("")
                if len(list_split) != 11:
                    list_split.insert(2, "")
                extract_array.append(list_split)

        clean_folder = f"/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Cleaned_Statement/"
        if not os.path.exists(clean_folder):
            os.makedirs(clean_folder)

        filter_folder = f"/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Filter_Statement/"
        if not os.path.exists(filter_folder):
            os.makedirs(filter_folder)
        
        acc_num = text_acc[17: 30]
        # Create dataframe and Save
        col_names = ["tran_date", "effective_date", "tran_code", "payment_amout","vat", "charge", "penalty", "int.(CR./DR.)", "int.(OS)","prin.(CR./DR.)", "prin.(OS)",]
        df = pd.DataFrame(extract_array, columns=col_names)
        df["tran_date"] = pd.to_datetime(df["tran_date"], format="%d/%m/%y")
        df["effective_date"] = pd.to_datetime(df["effective_date"], format="%d/%m/%y")
        df["tran_code"] = [x if x=="" else int(x) for x in df["tran_code"]]
        for col in col_names[3:]:
            list_ = []
            for text in df[col]:
                text = text.replace(',', '')
                idx = text.find('.')
                length = len(text[idx:])
                if length == 1:
                    text = text.replace('.', '')
                    list_.append(int(text))
                elif length == 2:
                    text = text.replace('.', '')
                    list_.append(int(text) / 10)
                elif length == 3:
                    text = text.replace('.', '')
                    list_.append(int(text) / 100)
            df[col] = list_

        df.to_excel(f'/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Cleaned_Statement/{file_dir[:-4]}' + f" {acc_num}.xlsx", index=False)

        con_prin = df['payment_amout'][0]
        df['%paid_amt_con_prin'] = [amt / con_prin for amt in df['payment_amout']]
        df = df[df['%paid_amt_con_prin'] >= 0.001]
        df.loc[:, 'contract_id'] = acc_num
        #df = df[df['prin.(CR./DR.)'] != 0]
        df.iloc[0, 3] = 0
        df.iloc[0, 9] = 0
        df.to_excel(f'/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Filter_Statement/filter_{file_dir[:-4]}' + f" {acc_num}.xlsx", index=False)
        df_combine = df_combine.append(df)

df_combine.to_excel(f'/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Filter_Statement/combine_filter.xlsx', index=False)

<ipython-input-2-7ee3aa3ef569>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'contract_id'] = acc_num
<ipython-input-2-7ee3aa3ef569>:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_combine = df_combine.append(df)
<ipython-input-2-7ee3aa3ef569>:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'contract_id'] = acc_num
<ipython-input-2-7ee3aa3ef569>:89: FutureWarning: The frame.append method i

In [ ]:
import os
from openpyxl import Workbook, load_workbook

dir_containing_files = '/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Filter_Statement'

dest_wb = Workbook()

for root, dir, filenames in os.walk(dir_containing_files):
    for file in filenames:
        file_name = file.split('.')[0]
        # Absolute Path for Excel files
        file_path = os.path.abspath(os.path.join(root, file))

        # Create new sheet in destination Workbook
        sheet_name=file_name[-13:]
        dest_wb.create_sheet(sheet_name)
        dest_ws = dest_wb[sheet_name]

        # Read source data
        source_wb = load_workbook(file_path)
        source_sheet = source_wb.active
        for row in source_sheet.rows:
            for cell in row:
                dest_ws[cell.coordinate] = cell.value

dest_wb.save('/content/drive/MyDrive/Colab Notebooks/Pinnacle/TCR text_extract/Cleaned_Statement/combine_filter_statement.xlsx')